In [1]:
%load_ext autoreload
%autoreload 2
import os
import matplotlib.pyplot as plt
import seaborn as sns
from os.path import join
from tqdm import tqdm
import pandas as pd
from typing import List
import numpy as np
from sasc.config import RESULTS_DIR
import sys
sys.path.append('..')
import json
EXPT_NAME = 'uts01_concepts_pilot_jun14'

r = (pd.read_pickle(join(RESULTS_DIR, 'sasc', 'fmri_results_merged.pkl'))
    .sort_values(by=['stability_score'], ascending=False))
r['id'] = "('" + r['top_explanation_init_strs'].str.replace(' ', '_').str.slice(stop=20) + "', '" + r['subject'] + "', " + r['module_num'].astype(str) + "),"    
r = r[r.subject == 'UTS01']

In [2]:
# commented out unstable voxels
D = {
    "temporal": [
        ('college days', 'UTS01', 378),
        ('specific times', 'UTS01', 157),
    ],
    "numeric": [
        ('numbers', 'UTS01', 458),
    ],
    "locational": [
        ('location', 'UTS01', 186),
        ('locations', 'UTS01', 100),
    ],
    "emotional": [
        ('embarrassment', 'UTS01', 322),
        # ('laughter or amusement', 'UTS01', 434),
        ('lonelineness', 'UTS01', 258),
    ],
    "social": [
        ('family relationships', 'UTS01', 192),  # same expl from both
        ('family and friends', 'UTS01', 107),
        # ('family and friends', 'UTS01', 365),
        # ('family and friends', 'UTS01', 204),
        ('communication', 'UTS01', 106),
        ('relationships', 'UTS01', 187),
        # ('relationships', 'UTS01', 122),
        ('communication', 'UTS01', 162),
        ('family and friends', 'UTS01', 332),
        # ('family and friends', 'UTS01', 340),
        # ('family and friends', 'UTS01', 374),
        # ('family and relationships', 'UTS01', 479),
        ('family relationships', 'UTS01', 144),
    ],
    "professional": [],
    # extra labels
    "movement": [
        ('movement', 'UTS01', 153),
        ('physical movement', 'UTS01', 39),
        # ('physical movement', 'UTS01', 489),
        ('physical contact', 'UTS01', 469),
        ('physical movement', 'UTS01', 171),
        ('movement', 'UTS01', 121),
    ],
    "manual": [
        ('vulgarities', 'UTS01', 311),
        ('violence and injury', 'UTS01', 232),
        ('food and drink', 'UTS01', 203),
        ('repeated words', 'UTS01', 484),
        ('food', 'UTS01', 473),
        ('clothing', 'UTS01', 149),
        ('speaking', 'UTS01', 451),
    ],
}
D_multi = {
    'multi': [
        ('identity and memory', 'UTS01', 304),
        ('food preparation', 'UTS01', 333),
        ('sneaking or avoiding', 'UTS01', 423),
        ('food and drink', 'UTS01', 52),
        ('deception', 'UTS01', 97),
    ],
}
json.dump(D, open(f"{EXPT_NAME}.json", "w"), indent=0)  # , indent=4)

In [3]:
D
vals = pd.DataFrame([tuple(x) for x in sum(list(D.values()), [])])
vals.columns = ["expl", "subject", "module_num"]
# display([val for val in vals['module_num'] if not val in r['module_num'].values])
for k in ["stability_score"]:
    vals["stability_score"] = vals.apply(
        lambda row: r[row["module_num"] == r["module_num"]].iloc[0][k],
        axis=1,
    )
vals = vals.sort_values(by="stability_score", ascending=True)

In [4]:
for i in range(len(vals)):
    row = vals.iloc[i]
    print(str(row['module_num']) + ', # ' +
          row['expl'] + f' ({row["stability_score"]:2f})')

322, # embarrassment (0.125209)
311, # vulgarities (0.132738)
258, # lonelineness (0.135058)
469, # physical contact (0.137269)
106, # communication (0.154657)
162, # communication (0.174878)
484, # repeated words (0.179070)
171, # physical movement (0.182128)
186, # location (0.184146)
378, # college days (0.199483)
458, # numbers (0.213608)
100, # locations (0.214551)
157, # specific times (0.217809)
451, # speaking (0.218006)
187, # relationships (0.221229)
332, # family and friends (0.233957)
121, # movement (0.235266)
149, # clothing (0.238112)
192, # family relationships (0.243154)
39, # physical movement (0.253595)
232, # violence and injury (0.266177)
107, # family and friends (0.285960)
153, # movement (0.286167)
144, # family relationships (0.298463)
203, # food and drink (0.313123)
473, # food (0.351168)


In [5]:
# display all rows
with pd.option_context(
    "display.max_rows", None, "display.max_columns", None, "display.max_colwidth", 200
):
    display(
        r.sort_values(by=["stability_score"], ascending=False)[
            [
                "id",
                "top_explanation_init_strs",
                "top_explanation_init_strs_llama",
                "explanation_init_strs",
                "stability_score",
                "fmri_test_corr",
                "frac_top_ngrams_module_correct",
                "top_score_normalized",
                "top_ngrams_module_correct",
            ]
        ]
        .round(3)
        .iloc[40:]
        .head(10)
    )

,id,top_explanation_init_strs,top_explanation_init_strs_llama,explanation_init_strs,stability_score,fmri_test_corr,frac_top_ngrams_module_correct,top_score_normalized,top_ngrams_module_correct
669,"('locations_or_movemen', 'UTS01', 463),",locations or movement between locations,presence of a mother figure,"[locations or movement between locations, locations, location]",0.219,0.321,0.627,0.989,"[in his backyard, into his backyard, in her attic, moved to brooklyn, took the subway, house in brooklyn, on the subway, ship to hoboken, live in brooklyn, house in manchester, landed in brooklyn,..."
106,"('speaking_or_making_s', 'UTS01', 451),",speaking or making some kind of sound,a question word (e.g,"[speaking or making some kind of sound, verbal communication, speaking or making a sound, communication]",0.218,0.367,0.867,0.601,"[smiled and said, smile and said, i said oh, said mama i, said did i, i said wow, screams fuck you, loudly said oh, i said mmm, said hey babe, and said well, i said listen, says hey dad, i said ye..."
1353,"('age_or_a_year_in_the', 'UTS01', 157),",age or a year in the past,food preparation,"[age or a year in the past, age, a specific year or age]",0.218,0.145,0.933,0.718,"[i was nineteen, i was fifteen, i was eleven, i was seventeen, i was twenty, i was fourteen, i was sixteen, me was nineteen, i was thirteen, i was nine, nineteen forty, in nineteen forty, i was tw..."
196,"('family_and_friends', 'UTS01', 289),",family and friends,physical action,[family and friends],0.218,0.210,0.933,2.605,"[my friends all, his friends all, my single friends, mom my sister, cousin my sister, friends come over, parents and friends, calls my brother, our friends come, told my family, friends are all, m..."
1238,"('locations', 'UTS01', 100),",locations,two words,"[locations, location]",0.215,0.227,0.680,0.339,"[on the windowsill, on the railing, onto the sidewalk, on the sidewalk, the bedroom window, across the couch, the windowsill, our bedroom window, sidewalk in front, over the embankment, at the roa..."
525,"('locations_or_items', 'UTS01', 460),",locations or items,"a form of the verb to be (was, were, is, are, been, be)","[locations or items, items or places that can be found in a city or urban environment, objects or activities that can be found in everyday life, objects or items found in a particular place, place...",0.214,0.227,0.893,2.366,"[go to mcdonald, into my purse, at the starbucks, cases of beer, line at mcdonald, box of twinkies, slices of bread, and shimmering skyscrapers, contain strip malls, pile of cds, lying on couches,..."
575,"('communication_or_int', 'UTS01', 86),",communication or interaction between two or more people,"fractions, such as half, nine percent, seventy five pence, half a dozen, eight so i, the three hundred, and the three top","[communication or interaction between two or more people, communication, communication, either verbal or nonverbal, laughter]",0.214,0.171,0.627,0.423,"[hear boys laughing, calls me and, she recognized me, called me from, called her back, called me and, approaches me and, leered at me, each other laughing, calls me later, call him crying, called ..."
1114,"('numbers_or_dates', 'UTS01', 458),",numbers or dates,"physical space, such as windows, doors, walls, rooms, closets, buildings, streets, and houses","[numbers or dates, numbers]",0.214,0.257,0.427,0.397,"[two thousand twenties, nine nineteen seventy, twentieth nineteen ninety, of nineteen fifty, five shillings five, fifty pounds fifty, fourteen ninety, dollar and fifty, forty silver twenty, kevin ..."
295,"('family_relationships', 'UTS01', 16),",family relationships,movement or direction,"[family relationships, family]",0.213,0.317,0.827,1.221,"[calls my brother, cousin my sister, mom my sister, tell my mom, tells my wife, tell their dad, to his mom, illness my sister, tell her sisters, call my mother, cousin whose mother, splash her bro..."
1071,"('a_reference_to_a_spe', 'UTS01', 442),","a reference to a speci

In [ ]:
# with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', 200):
#     display(r[['id', 'top_explanation_init_strs', 'top_explanation_init_strs_llama']].sort_values(by='stability_score').transpose())